In [1]:
using DataFrames

In [2]:
cur_table = readtable("../data/tokamak_db.csv")

cur_table[:SELDB3] = map(Int,cur_table[:SELDB3])

return

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at /Users/dan/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [3] readtable(::String) at /Users/dan/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] softscope_include_string(::Module, ::String, ::String) at /Users/dan/.julia/v0.6/SoftGlobalScope/src/SoftGlobalScope.jl:66
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/dan/.julia/v0.6/IJulia/src/execute_request.jl:193
 [7] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/dan/.julia/v0.6/Compat/src/Compat.jl:189
 [8] eventloop(::ZMQ.Socket) at /Users/dan/.julia/v0.6/I

In [3]:
cur_func = function(aa,nn,ii,rr,bb,AA,PL,kk,ss)
    ( ss == 1111111111 ) || return NaN
    
    ismissing(aa) && return NaN
    ismissing(nn) && return NaN
    ismissing(ii) && return NaN
    ismissing(rr) && return NaN
    ismissing(bb) && return NaN
    ismissing(AA) && return NaN
    ismissing(PL) && return NaN
    ismissing(kk) && return NaN
    ismissing(ss) && return NaN
    
    cur_val = NaN
    try
        cur_val = ( 0.145 * (abs(ii)/1e6)^0.93 * rr ^ 1.39 * aa ^ 0.58 * kk ^ 0.78 * (nn/1e20) ^ 0.41 * abs(bb) ^ 0.15 * AA ^ 0.19 / (PL/1e6) ^ 0.69)
        ismissing(cur_val) && return NaN
        isfinite(cur_val) || return NaN
        @assert cur_val > 0
        
        return cur_val
    catch cur_error
        println(cur_error)
        println(cur_val)
        println((aa,nn,ii,rr,bb,AA,PL,kk,ss))
        return NaN
    end
end

cur_table[:TAUE] = map(
    cur_func, 
    cur_table[:AMIN], cur_table[:NEL], cur_table[:IP], 
    cur_table[:RMAG], cur_table[:BT], cur_table[:MEFF], 
    cur_table[:PL], cur_table[:KAREA], cur_table[:SELDB3]
)

cur_table[:NG] = map(
    (aa,nn,ii) -> ( pi*aa^2*nn/(1.e14*abs(ii)) ), 
    cur_table[:AMIN], cur_table[:NEL], cur_table[:IP]
)

cur_table[:PREAL] = map(
    (ppl,pprad) -> ( ppl - pprad ), 
    cur_table[:PL], cur_table[:PRAD]
)

cur_table[:EPSILON] = map(
    (rmag,amin) -> ( amin / rmag ), 
    cur_table[:RMAG], cur_table[:AMIN]
)


return

In [9]:
Pkg.add("Query")
Pkg.add("MultivariateStats")

INFO: Package Query is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Query
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Installing MultivariateStats v0.5.0
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of MultivariateStats
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [10]:
using Interact
using DataStructures

using StringCases
using LaTeXStrings
using Query

using Plots
pgfplots()

using MultivariateStats
# plotlyjs()

INFO: Recompiling stale cache file /Users/dan/.julia/lib/v0.6/TableTraits.ji for module TableTraits.
INFO: Recompiling stale cache file /Users/dan/.julia/lib/v0.6/IterableTables.ji for module IterableTables.
INFO: Recompiling stale cache file /Users/dan/.julia/lib/v0.6/DataValues.ji for module DataValues.
INFO: Recompiling stale cache file /Users/dan/.julia/lib/v0.6/QueryOperators.ji for module QueryOperators.
INFO: Recompiling stale cache file /Users/dan/.julia/lib/v0.6/MultivariateStats.ji for module MultivariateStats.


In [11]:
cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @where 0 < cur_row.NG < 2.5
    @where 0 < cur_row.TEV
    @select cur_row.TAUE
    @collect
end

cur_fit_tau = Array(cur_query)

cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @where 0 < cur_row.NG < 2.5
    @where 0 < cur_row.TEV
    @select cur_row.TAUTOT
    @collect
end

cur_table_tau = Array(cur_query)

cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @where 0 < cur_row.NG < 2.5
    @where 0 < cur_row.TEV
    @select cur_row.NG
    @collect
end

cur_N_G_list = sort(Array(cur_query))

cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @where 0 < cur_row.NG < 2.5
    @where 0 < cur_row.TEV
    @select cur_row.TOK
    @collect
end

cur_tok_list = sort(Array(cur_query))

cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @where 0 < cur_row.NG < 2.5
    @where 0 < cur_row.TEV
    @select cur_row.AMIN, cur_row.NEL, cur_row.IP, cur_row.RMAG, cur_row.BT, 
cur_row.MEFF, cur_row.PL, cur_row.KAREA, cur_row.SELDB3, cur_row.PL, cur_row.WTOT, cur_row.PRAD, cur_row.PREAL, 
    cur_row.TEV, cur_row.EPSILON
    @collect
end

work_keys = [:AMIN, :NEL, :IP, :RMAG, :BT, :MEFF, :PL, :KAREA, :SELDB3, :PL, :WTOT, :PRAD, :PREAL, :TEV, :EPSILON ]
work_dict = Dict()

work_dict[:NG] = cur_N_G_list
work_dict[:table_tau] = cur_table_tau
work_dict[:fit_tau] = cur_fit_tau

cur_array = Array(cur_query)

for (cur_index, cur_key) in enumerate(work_keys)
    
    cur_vector = map(cur_row -> cur_row[cur_index], cur_array)
    
    work_dict[cur_key] = map(abs, cur_vector)
    
    any(cur_element -> isa(cur_element, DataValues.DataValue), cur_vector) || continue
    
    if cur_key == :PREAL
        cur_vector = map(cur_element -> ismissing(cur_element) ? cur_element : 0.0, cur_vector)
    elseif cur_key == :EPSILON
        @assert all(isfinite, map(get,cur_vector))
    else
        cur_vector = map(cur_element -> isfinite(cur_element) ? cur_element : 0.0, cur_vector)
    end
    
    work_dict[cur_key] = cur_vector
    
    any(cur_element -> isa(cur_element, DataValues.DataValue), cur_vector) || continue
    
    cur_vector = map(float, cur_vector)
    
    work_dict[cur_key] = cur_vector

    if cur_key == :NEL
        work_dict[cur_key] ./= 1e20
    elseif cur_key == :IP || cur_key == :PL || cur_key == :PRAD || cur_key == :PL || cur_key == :WTOT || cur_key == :PREAL
        work_dict[cur_key] ./= 1e6
    elseif cur_key == :TEV
        work_dict[cur_key] ./= 1e3
    else
    end
end

for (cur_key, cur_value) in work_dict
    work_dict[cur_key] = map(
        xx -> ( try return get(xx) catch return xx end ),
        cur_value
    )
    
    work_dict[cur_key] = map(abs, work_dict[cur_key])
end

for (cur_index, (cur_PL, cur_PREAL)) in enumerate(zip(work_dict[:PL], work_dict[:PREAL]))
    iszero(cur_PREAL) || continue
    work_dict[:PREAL][cur_index] = cur_PL
end

return

In [12]:
cur_plot = scatter(cur_table_tau, cur_fit_tau, opacity=0.4, markersize=0.65, label="")

cur_min = min(minimum(cur_fit_tau), minimum(cur_table_tau)) / 1.4
cur_max = max(maximum(cur_fit_tau), maximum(cur_table_tau)) * 1.4

plot!([cur_min, cur_max], [cur_min, cur_max], label="")

plot!(aspectratio=:equal, xscale=:log10, yscale=:log10)
xlabel!(L"(\tau_E)_{ TABLE } \ \ [ \textnormal{s} ]")
ylabel!(L"(\tau_E)_{ FIT } \ \ [ \textnormal{s} ]")

title!("Confinement Time Scaling Fit")

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 3.296875 2.390625 C 3.296875 2.359375 3.296875 2.34375 3.125 2.171875 C 1.890625 0.921875 1.5625 -0.96875 1.5625 -2.5 C 1.5625 -4.234375 1.9375 -5.96875 3.171875 -7.203125 C 3.296875 -7.328125 3.296875 -7.34375 3.296875 -7.375 C 3.296875 -7.453125 3.265625 -7.484375 3.203125 -7.484375 C 3.09375 -7.484375 2.203125 -6.796875 1.609375 -5.53125 C 1.109375 -4.4375 0.984375 -3.328125 0.984375 -2.5 C 0.984375 -1.71875 1.09375 -0.515625 1.640625 0.625 C 2.25 1.84375 3.09375 2.5 3.203125 2.5 C 3.265625 2.5 3.296875 2.46875 3.296875 2.390625 Z M 3.296875 2.390625 "/>
 
 
<path style="stroke:none;" d="M 2.875 -2.5 C 2.875 -3.265625 2.765625 -4.46875 2.21875 -5.609375 C 1.625 -6.828125 0.765625 -7.484375 0.671875 -7.484375 C 0.609375 -7.484375 0.5625 -7.4375 0.5625 -7.375 C 0.5625 -7.34375 0.5625 -7.328125 0.75 -7.140625 C 1.734375 -6.15625 2.296875 -4.578125 2.296875 -2.5 C 2.296875 -0.78125 1.9375 0.96875 0.703125 2.21875 C 0.5625 2.34375 0.5625 2.359375 0.5625 2.390625 C 0.5625 2.453125 0.609375 2.5 0.671875 2.5 C 0.765625 2.5 1.671875 1.8125 2.25 0.546875 C 2.765625 -0.546875 2.875 -1.65625 2.875 -2.5 Z M 2.875 -2.5 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 3.515625 -1.265625 L 3.28125 -1.265625 C 3.265625 -1.109375 3.1875 -0.703125 3.09375 -0.640625 C 3.046875 -0.59375 2.515625 -0.59375 2.40625 -0.59375 L 1.125 -0.59375 C 1.859375 -1.234375 2.109375 -1.4375 2.515625 -1.765625 C 3.03125 -2.171875 3.515625 -2.609375 3.515625 -3.265625 C 3.515625 -4.109375 2.78125 -4.625 1.890625 -4.625 C 1.03125 -4.625 0.4375 -4.015625 0.4375 -3.375 C 0.4375 -3.03125 0.734375 -2.984375 0.8125 -2.984375 C 0.96875 -2.984375 1.171875 -3.109375 1.171875 -3.359375 C 1.171875 -3.484375 1.125 -3.734375 0.765625 -3.734375 C 0.984375 -4.21875 1.453125 -4.375 1.78125 -4.375 C 2.484375 -4.375 2.84375 -3.828125 2.84375 -3.265625 C 2.84375 -2.65625 2.40625 -2.1875 2.1875 -1.9375 L 0.515625 -0.265625 C 0.4375 -0.203125 0.4375 -0.1875 0.4375 0 L 3.3125 0 Z M 3.515625 -1.265625 "/>
 
 
<path style="stroke:none;" d="M 3.59375 -2.21875 C 3.59375 -2.984375 3.5 -3.546875 3.1875 -4.03125 C 2.96875 -4.34375 2.53125 -4.625 1.984375 -4.625 C 0.359375 -4.625 0.359375 -2.71875 0.359375 -2.21875 C 0.359375 -1.71875 0.359375 0.140625 1.984375 0.140625 C 3.59375 0.140625 3.59375 -1.71875 3.59375 -2.21875 Z M 1.984375 -0.0625 C 1.65625 -0.0625 1.234375 -0.25 1.09375 -0.8125 C 1 -1.21875 1 -1.796875 1 -2.3125 C 1 -2.828125 1 -3.359375 1.09375 -3.734375 C 1.25 -4.

In [13]:
cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @select cur_row.TAUE
    @collect
end

cur_fit_tau = map(log10, Array(cur_query))

cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @select cur_row.TAUTOT
    @collect
end

cur_table_tau = map(log10, Array(cur_query))

cur_plot = scatter(cur_table_tau, cur_fit_tau, opacity=0.4, markersize=0.65, label="")

cur_min = -2.1
cur_max = 0.35

ylims!(cur_min, cur_max)

plot!([cur_min, cur_max], [cur_min, cur_max], label="", xticks = ([-2,-1.5,-1.0,-0.5,0.0],["0.01","","0.1","","1.0"]), yticks = ([-2,-1.5,-1.0,-0.5,0.0],["0.01","","0.1","","1.0"]))

plot!(aspectratio=:equal)
xlabel!(L"(\tau_E)_{ TABLE } \ \ [ \textnormal{s} ]")
ylabel!(L"(\tau_E)_{ FIT } \ \ [ \textnormal{s} ]")

title!("Confinement Time Scaling Fit")

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
 
 
 
<path style="stroke:none;" d="M 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 L 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 Z M 4.171875 0 "/>
 
 
<path style="stroke:none;" d="M 3.59375 -1.28125 C 3.59375 -1.796875 3.296875 -2.109375 3.171875 -2.21875 C 2.84375 -2.546875 2.453125 -2.625 2.03125 -2.703125 C 1.46875 -2.8125 0.8125 -2.9375 0.8125 -3.515625 C 0.8125 -3.875 1.0625 -4.28125 1.921875 -4.28125 C 3.015625 -4.28125 3.078125 -3.375 3.09375 -3.078125 C 3.09375 -2.984375 3.203125 -2.984375 3.203125 -2.984375 C 3.34375 -2.984375 3.34375 -3.03125 3.34375 -3.21875 L 3.34375 -4.234375 C 3.34375 -4.390625 3.34375 -4.46875 3.234375 -4.46875 C 3.1875 -4.46875 3.15625 -4.46875 3.03125 -4.34375 C 3 -4.3125 2.90625 -4.21875 2.859375 -4.1875 C 2.484375 -4.46875 2.078125 -4.46875 1.921875 -4.46875 C 0.703125 -4.46875 0.328125 -3.796875 0.328125 -3.234375 C 0.328125 -2.890625 0.484375 -2.609375 0.75 -2.390625 C 1.078125 -2.140625 1.359375 -2.078125 2.078125 -1.9375 C 2.296875 -1.890625 3.109375 -1.734375 3.109375 -1.015625 C 3.109375 -0.515625 2.765625 -0.109375 1.984375 -0.109375 C 1.140625 -0.109375 0.78125 -0.671875 0.59375 -1.53125 C 0.5625 -1.65625 0.5625 -1.6875 0.453125 -1.6875 C 0.328125 -1.6875 0.328125 -1.625 0.328125 -1.453125 L 0.328125 -0.125 C 0.328125 0.046875 0.328125 0.109375 0.4375 0.109375 C 0.484375 0.109375 0.5 0.09375 0.6875 -0.09375 C 0.703125 -0.109375 0.703125 -0.125 0.890625 -0.3125 C 1.328125 0.09375 1.78125 0.109375 1.984375 0.109375 C 3.125 0.109375 3.59375 -0.5625 3.59375 -1.28125 Z M 3.59375 -1.28125 "/>
 
 
<path style="stroke:none;" d="M 6.625 -2.328125 C 6.625 -2.421875 6.625 -2.5 6.5 -2.5 C 6.390625 -2.5 6.390625 -2.4375 6.375 -2.328125 C 6.296875 -0.90625 5.234375 -0.09375 4.140625 -0.09375 C 3.53125 -0.09375 1.578125 -0.421875 1.578125 -3.40625 C 1.578125 -6.375 3.53125 -6.71875 4.140625 -6.71875 C 5.21875 -6.71875 6.109375 -5.8125 6.3125 -4.359375 C 6.328125 -4.21875 6.328125 -4.1875 6.46875 -4.1875 C 6.625 -4.1875 6.625 -4.21875 6.625 -4.421875 L 6.625 -6.78125 C 6.625 -6.953125 6.625 -7.03125 6.515625 -7.03125 C 6.484375 -7.03125 6.4375 -7.03125 6.359375 -6.90625 L 5.859375 -6.171875 C 5.5 -6.53125 4.984375 -7.03125 4.03125 -7.03125 C 2.15625 -7.03125 0.5625 -5.4375 0.5625 -3.40625 C 0.5625 -1.34375 2.171875 0.21875 4.03125 0.21875 C 5.65625 0.21875 6.625 -1.171875 6.625 -2.328125 Z M 6.625 -2.328125 "/>
 
 
<path style="stroke:none;" d="M 4.6875 -2.140625 C 4.6875 -3.40625 3.703125 -4.46875 2.5 -4.46875 C 1.25 -4.46875 0.28125 -3.375 0.28125 -2.140625 C 0.28125 -0.84375 1.3125 0.109375 2.484375 0.109375 C 3.6875 0.109375 4.6875 -0.875 4.6875 -2.14062

In [ ]:
savefig(cur_plot, "h_mode.tex")

In [14]:
histogram((actual_values-expected_values)./expected_values)
ylims!(0,500)
# xlims!(-1.0,1.5)

LoadError: [91mUndefVarError: actual_values not defined[39m

In [15]:
histogram((work_dict[:fit_tau]-work_dict[:table_tau])./work_dict[:table_tau])
ylims!(0,500)
xlims!(-1.0,1.5)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 L 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 Z M 4.171875 0 "/>
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.109375 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 Z M 4.46875 -1.734375 "/>
 
 
<path style="stroke:none;" d="M 4.5625 -1.703125 C 4.5625 -2.515625 3.921875 -3.296875 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765

In [60]:
used_keys = [:PREAL, :BT, :RMAG, :KAREA, :AMIN, :MEFF, :IP, :TEV]
used_keys = [:PREAL, :BT, :RMAG, :KAREA, :AMIN, :MEFF, :IP]

7-element Array{Symbol,1}:
 :PREAL
 :BT   
 :RMAG 
 :KAREA
 :AMIN 
 :MEFF 
 :IP   

In [61]:
cur_X = []
cur_Y = Float64[]
for cur_entry in 1:length(first(values(work_dict)))
#     push!(cur_Y, log(work_dict[:WTOT][cur_entry]/(work_dict[:PL][cur_entry])))
        
    push!(cur_Y, log(work_dict[:WTOT][cur_entry]/(work_dict[:PREAL][cur_entry])))
    
    tmp_X = Float64[]
    for cur_key in used_keys
        push!(tmp_X, log(work_dict[cur_key][cur_entry]))
    end
    
    push!(cur_X, tmp_X)
end

In [78]:
function cur_tau(cur_reg)
    cur_A = llsq(hcat(cur_X...)', cur_Y)#, cur_reg)
    
    ridge_a, ridge_b = cur_A[1:end-1], cur_A[end]
    
    ridge_a = map(zz -> round(zz,2), ridge_a)
    
    ridge_b = signif(exp(ridge_b),1)
    
    println(" > ", ridge_b)
    
    for (cur_index, cur_key) in enumerate(used_keys)
        println(cur_key, " - ", ridge_a[cur_index])
    end
    
    cur_function = function(PL, BT, RMAG, KAREA, AMIN, MEFF, IP)#, TEV)
        cur_tau = ridge_b
        
        cur_tau *= PL ^ ridge_a[1]
        cur_tau *= BT ^ ridge_a[2]
        cur_tau *= RMAG ^ ridge_a[3]
        cur_tau *= KAREA ^ ridge_a[4]
        cur_tau *= AMIN ^ ridge_a[5]
        cur_tau *= MEFF ^ ridge_a[6]
        cur_tau *= IP ^ ridge_a[7]
#         cur_tau *= TEV ^ ridge_a[8]
        
        cur_tau
    end
    
    cur_function
end


cur_tau (generic function with 1 method)

In [79]:
work_dict

Dict{Any,Any} with 17 entries:
  :PL        => [0.6206, 0.4984, 0.6161, 0.4973, 0.5685, 0.6145, 0.6358, 0.3762…
  :WTOT      => [0.01014, 0.0096, 0.0091, 0.0104, 0.008279, 0.01024, 0.01087, 0…
  :BT        => [1.976, 1.972, 1.982, 1.97, 1.952, 1.963, 1.952, 1.951, 2.42, 2…
  :table_tau => [0.01634, 0.01926, 0.01477, 0.02091, 0.01456, 0.01666, 0.0171, …
  :IP        => [0.2199, 0.22, 0.2209, 0.2191, 0.1595, 0.2198, 0.2199, 0.2198, …
  :PRAD      => [0.1061, 0.1079, 0.1221, 0.1342, 0.1251, 0.1157, 0.1123, 0.0847…
  :PREAL     => [0.6206, 0.4984, 0.6161, 0.4973, 0.5685, 0.6145, 0.6358, 0.3762…
  :NG        => [0.147639, 0.16818, 0.169635, 0.175448, 0.177042, 0.17762, 0.17…
  :AMIN      => [0.2124, 0.2139, 0.212, 0.2132, 0.2108, 0.2151, 0.2156, 0.2153,…
  :MEFF      => [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0  …  2.0, 2.0,…
  :fit_tau   => [0.0164142, 0.0189667, 0.0160492, 0.0196876, 0.0125232, 0.01661…
  :EPSILON   => [0.25154, 0.252717, 0.251602, 0.25198, 0.244888, 0.252732, 0.2

In [80]:
used_keys

7-element Array{Symbol,1}:
 :PREAL
 :BT   
 :RMAG 
 :KAREA
 :AMIN 
 :MEFF 
 :IP   

In [81]:
expected_values = work_dict[:WTOT]./(work_dict[:PREAL])

actual_values = nothing

cur_space = logspace(-3,0,5)
# @manipulate for cur_index=1:length(cur_space)
cur_index=1
    cur_reg = cur_space[cur_index]
cur_fit = cur_tau(cur_reg)
    
    println(cur_reg)

actual_values = []

for cur_entry in 1:length(first(values(work_dict)))
    cur_input = Float64[]
    for cur_key in used_keys
        push!(cur_input, work_dict[cur_key][cur_entry])
    end
    push!(actual_values, cur_fit(cur_input...))
end

# scatter(expected_values, actual_values)
# # scatter!(cur_table_tau, cur_fit_tau)
    
# # histogram((work_dict[:fit_tau]-work_dict[:table_tau])./work_dict[:table_tau])
# # ylims!(0.2,2)
# # xlims!(0.1,1.5)
# plot!(xscale=:log10, yscale=:log10)
    
# plot!(aspectratio=:equal, opacity=0.4, markersize=0.65, label="")
# xlabel!(L"(\tau_E)_{ TABLE } \ \ [ \textnormal{s} ]")
# ylabel!(L"(\tau_E)_{ FIT } \ \ [ \textnormal{s} ]")

#     display(mean(expected_values)/mean(actual_values))
# title!("Confinement Time Scaling Fit")
# #     ylims!(1e-2,1e1)
#     ylabel!("$(cur_reg)")
    
#     xlims!(0.01, 2)
#         ylims!(0.01, 2)
#     plot!([0.01,2],[0.01,2])
    tmp_plot = plot()
    histogram!(abs(actual_values-expected_values)./expected_values, label="New Scaling", bins=40)
# ylims!(0,500)
# xlims!(-1.0,1.5)
    
    histogram!(abs(work_dict[:fit_tau]-work_dict[:table_tau])./work_dict[:table_tau], opacity=0.5, label="ITER 98", bins=20)
# xlims!(0,1.0)
#    ylims!(0,1) 
    xlabel!("Relative Error")
    ylabel!("Count")
    
    xlims!(0,1)
    savefig(tmp_plot, "new_scaling.tex")
    
    title!("Confinement Scalings")
# end

 > 0.08
PREAL - -0.54
BT - 0.3
RMAG - 1.32
KAREA - 0.56
AMIN - 0.17
MEFF - 0.23
IP - 0.93
0.001


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,1}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] softscope_include_string(::Module, ::String, ::String) at /Users/dan/.julia/v0.6/SoftGlobalScope/src/SoftGlobalScope.jl:66
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/dan/.julia/v0.6/IJulia/src/execute_request.jl:193
 [6] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/dan/.julia/v0.6/Compat/src/Compat.jl:189
 [7] eventloop(::ZMQ.Socket) at /Users/dan/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##13#16)() at ./task.jl:335
while loading In[81], in expression starting on line 44
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,1}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] softscope_include_string(::Module, ::String, ::String) at /Users/dan/.julia/v

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
 
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 Z M 4.46875 -1.734375 "/>
 
 
<path style="stroke:none;" d="M 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 L 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96875 0 C 2.375 -0.03125 2.890625 -0.03125 3.3125 -0.03125 C 3.734375 -0.03125 4.25 -0.03125 4.671875 0 L 4.671875 -0.3125 L 4.453125 -0.3125 C 3.71875 -0.3125 3.703125 -0.421875 3.703125 -0.78125 L 3.703125 -1.640625 Z M 2.984375 -1.953125 L 0.5625 -1.953125 L 2.984375 -5.671875 Z M 2.984375 -1.953125 "/>
 
 
<path style="stroke:none;" d="M 4.5625 -2.03125 C 4.5625 -3.296875 3.671875 -4.25 2.5625 -4.25 C 1.890625 -4.25 1.515625 -3.75 1.3125 -3.265625 L 1.3125 -3.515625 C 1.3125 -6.03125 2.546875 -6.390625 3.0625 -6.390625 C 3.296875 -6.390625 3.71875 -6.328125 3.9375 -5.984375 C 3.78125 -5.984375 3.390625 -5.984375 3.390625 -5.546875 C 3.390625 -5.234375 3.625 -5.078125 3.84375 -5.078125 C 4 -5.078125 4.3125 -5.171875 4.3125 -5.5625 C 4.3125 -6.15625 3.875 -6.640625 3.046875 -6.640625 C 1.765625 -6.640625 0.421875 -5.359375 0.421875 -3.15625 C 0.421875 -0.484375 1.578125 0.21875 2.5 0.21875 C 3.609375 0.21875 4.5625 -0.71875 4.5625 -2.03125 Z M 3.65625 -2.046875 C 3.65625 -1.5625 3.65625 -1.0625 3.484375 -0.703125 C 3.1875 -0.109375 2.734375 -0.0625 2.5 -0.0625 C 1.875 -0.0625 1.578125 -0.65625 1.515625 -0.8125 C 1.328125 -1.28125 1.328125 -2.078125 1.328125 -2.25 C 1.328125 -3.03125 1.65625 -4.03125 2.546875 -4.03125 C 2.71875 -4.03125 3.171875 -4.03125 3.484375 -3.40625 C 3.65625 -3.046875 3.65625 -2.53125 3.65625 -2.046875 Z M 3.65625 -2.046875 "/>
 
 
<path style="stroke:none;" d="M 4.5625 -1.671875 C 4.5625 -2.03125 4.453125 -2.484375 4.0625 -2.90625 C 3.875 -3.109375 3.71875 -3.203125 3.078125 -3.609375 C 3.796875 -3.984375 4.28125 -4.5 4.28125 -5.15625 C 4.28125 -6.078125 3.40625 -6.640625 2.5 -6.640625 C 1.5 -6.640625 0.6875 -5.90625 0.6875 -4.96875 C 0.6875 -4.796875 0.703125 -4.34375 1.125 -3.875 C 1.234375 -3.765625 1.609375 -3.515625 1.859375 -3.34375 C 1.28125 -3.046875 0.421875 -2.

In [56]:
using DataStructures

In [21]:
log(A_min) / log(A_max)

LoadError: [91mUndefVarError: A_min not defined[39m

In [22]:
for (cur_key, cur_value) in work_dict
    println(cur_key) 
    
    println(maximum(log.(cur_value))-minimum(log.(cur_value)))
    
end

PL
4.190492114657845
WTOT
7.402631212956019
BT
2.017359420696588
table_tau
4.559500149154903
IP
3.471596439206187
PRAD
Inf
PREAL
4.190492114657845
NG
2.0592679262294937
AMIN
1.6834781132482957
MEFF
1.094604267270571
fit_tau
4.622321848551985
EPSILON
0.904259749941069
SELDB3
0.0
NEL
3.5628881537627013
TEV
4.604233417769705
KAREA
0.9252631016621206
RMAG
1.6120477442764711


In [23]:
0.008433787956526195*0.5831208188930297*1.049901410970401*1.7061316228070864*1.0191494521259126*1.498909609837451*1.2855573280954944*5.319690847684186*1.0

0.09203085113953469

In [47]:
used_keys = [:EPSILON, :MEFF, :TEV]

solved_dict = OrderedDict(
    :IP => 0.7,
    :PREAL => -0.5,
    :RMAG => 0.5,
    :BT => 0.1,
    :KAREA => 0.1
)

expected_values = work_dict[:WTOT]./work_dict[:PREAL]

function log_work_scalar(cur_value, cur_key, cur_array=nothing)
    ( cur_array == nothing ) && ( cur_array = work_dict[cur_key] )
    ( log(cur_value) - minimum(log.(cur_array)) ) / ( maximum(log.(cur_array)) - minimum(log.(cur_array)) )
end

function lin_work_scalar(cur_value, cur_key, cur_exp, cur_array=nothing)
    ( cur_array == nothing ) && ( cur_array = work_dict[cur_key] )
    
    real_exp = cur_exp 
    
    real_exp /= log(maximum(cur_array)/minimum(cur_array))
    
    real_exp *= log(maximum(expected_values)/minimum(expected_values)) 
    
#     real_exp = round(real_exp, 2)
    
#     println("exp ", real_exp)
    
    cur_answer = ( cur_value / minimum(cur_array) ) ^ real_exp
    
#     new_factor = round(new_factor, 5)
    
#     println("fac ", new_factor)
    
    cur_answer
end

cur_X = []
cur_Y = Float64[]
for cur_entry in 1:length(first(values(work_dict)))
#     push!(cur_Y, log(work_dict[:WTOT][cur_entry]/(work_dict[:PL][cur_entry])))
        
    tmp_Y = log_work_scalar(expected_values[cur_entry], nothing, expected_values)
    
    for (cur_key, cur_value) in solved_dict
        tmp_Y -= log_work_scalar(work_dict[cur_key][cur_entry], cur_key) * cur_value
    end
    
    push!(cur_Y, tmp_Y)
        
    tmp_X = Float64[]
    for cur_key in used_keys
        push!(tmp_X, log_work_scalar(work_dict[cur_key][cur_entry], cur_key))
    end
    
    push!(cur_X, tmp_X)
end

function cur_tau(cur_reg)
    cur_A = llsq(hcat(cur_X...)', cur_Y)#, cur_reg)
    
    ridge_a, ridge_b = cur_A[1:end-1], cur_A[end]
    
#     ridge_a = map(zz -> iszero(round(zz,1)) ? round(zz,2) : round(zz,1), ridge_a)
    
    ridge_b = exp(ridge_b)
    
    ridge_b = signif(ridge_b,3)
    
    println(ridge_b)
    
    println(ridge_a)
    
    ridge_b ^= ( maximum(log.(expected_values)) - minimum(log.(expected_values)) )
    
    ridge_b *= minimum(expected_values)
    
#     ridge_b = round(ridge_b, 10)
    
    println("> ", ridge_b)
            
    cur_function = function(EPSILON, MEFF, TEV, IP, PREAL, RMAG, BT, KAREA)
        cur_tau = ridge_b
        
        tmp_dict = OrderedDict(
            :EPSILON => EPSILON,
            :MEFF => MEFF,
            :TEV => TEV
        )
        
        for (tmp_index, (tmp_key, tmp_value)) in enumerate(tmp_dict)
#             println(tmp_key, " _ ", tmp_value, " - ", ridge_a[tmp_index])
            cur_tau *= lin_work_scalar(tmp_value, tmp_key,ridge_a[tmp_index] )
#             cur_tau *= work_scalar(tmp_value, tmp_key) ^ ridge_a[tmp_index]
        end
        
        tmp_dict = OrderedDict(
            :IP => IP,
            :PREAL => PREAL,
            :RMAG => RMAG,
            :BT => BT,
            :KAREA => KAREA
        )
        
        for (tmp_index, (tmp_key, tmp_value)) in enumerate(tmp_dict)
#             println(tmp_key, " _ ", tmp_value, " - ", ridge_a[tmp_index])
            cur_tau *= lin_work_scalar(tmp_value, tmp_key,solved_dict[tmp_key] )
#             cur_tau *= work_scalar(tmp_value, tmp_key) ^ ridge_a[tmp_index]
        end
        
        cur_tau
    end
    
    cur_function
end

actual_values = nothing

cur_space = logspace(-1,6,21)
# @manipulate for cur_index=1:length(cur_space)
cur_index=1
    cur_reg = cur_space[cur_index]
cur_fit = cur_tau(cur_reg)
    
actual_values = []

for cur_entry in 1:length(first(values(work_dict)))
    cur_input = Float64[]
    for cur_key in used_keys
        push!(cur_input, work_dict[cur_key][cur_entry])
    end
    for (cur_key, cur_value) in solved_dict
        push!(cur_input, work_dict[cur_key][cur_entry])
    end
    push!(actual_values, cur_fit(cur_input...))
end

# scatter(expected_values, actual_values)
# # scatter!(cur_table_tau, cur_fit_tau)
    
# # histogram((work_dict[:fit_tau]-work_dict[:table_tau])./work_dict[:table_tau])
# # ylims!(0.2,2)
# # xlims!(0.1,1.5)
# plot!(xscale=:log10, yscale=:log10)
    
# plot!(aspectratio=:equal, opacity=0.4, markersize=0.65, label="")
# xlabel!(L"(\tau_E)_{ TABLE } \ \ [ \textnormal{s} ]")
# ylabel!(L"(\tau_E)6_{ FIT } \ \ [ \textnormal{s} ]")

#     display(mean(expected_values)/mean(actual_values))
# title!("Confinement Time Scaling Fit")
# #     ylims!(1e-2,1e1)
#     ylabel!("$(cur_reg)")
    
#     xlims!(0.01, 2)
#         ylims!(0.01, 2)
#     plot!([0.01,2],[0.01,2])
    tmp_plot = plot()
    histogram!(abs(actual_values-expected_values)./expected_values, label="New Scaling", bins=80)
# ylims!(0,500)
# xlims!(-1.0,1.5)
    
    histogram!(abs(work_dict[:fit_tau]-work_dict[:table_tau])./work_dict[:table_tau], opacity=0.5, label="ITER 98", bins=20)
# xlims!(0,1.0)
#    ylims!(0,1) 
    xlabel!("Relative Error")
    ylabel!("Count")
    
    xlims!(0,1)
    savefig(tmp_plot, "new_scaling.tex")
    
    title!("Confinement Scalings")
# end

0.889
[0.0386533, 0.0583259, 0.116949]
> 0.00851704717785172


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,1}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] softscope_include_string(::Module, ::String, ::String) at /Users/dan/.julia/v0.6/SoftGlobalScope/src/SoftGlobalScope.jl:66
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/dan/.julia/v0.6/IJulia/src/execute_request.jl:193
 [6] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/dan/.julia/v0.6/Compat/src/Compat.jl:189
 [7] eventloop(::ZMQ.Socket) at /Users/dan/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##13#16)() at ./task.jl:335
while loading In[47], in expression starting on line 161
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,1}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] softscope_include_string(::Module, ::String, ::String) at /Users/dan/.julia/

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
 
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 Z M 4.46875 -1.734375 "/>
 
 
<path style="stroke:none;" d="M 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 L 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96875 0 C 2.375 -0.03125 2.890625 -0.03125 3.3125 -0.03125 C 3.734375 -0.03125 4.25 -0.03125 4.671875 0 L 4.671875 -0.3125 L 4.453125 -0.3125 C 3.71875 -0.3125 3.703125 -0.421875 3.703125 -0.78125 L 3.703125 -1.640625 Z M 2.984375 -1.953125 L 0.5625 -1.953125 L 2.984375 -5.671875 Z M 2.984375 -1.953125 "/>
 
 
<path style="stroke:none;" d="M 4.5625 -2.03125 C 4.5625 -3.296875 3.671875 -4.25 2.5625 -4.25 C 1.890625 -4.25 1.515625 -3.75 1.3125 -3.265625 L 1.3125 -3.515625 C 1.3125 -6.03125 2.546875 -6.390625 3.0625 -6.390625 C 3.296875 -6.390625 3.71875 -6.328125 3.9375 -5.984375 C 3.78125 -5.984375 3.390625 -5.984375 3.390625 -5.546875 C 3.390625 -5.234375 3.625 -5.078125 3.84375 -5.078125 C 4 -5.078125 4.3125 -5.171875 4.3125 -5.5625 C 4.3125 -6.15625 3.875 -6.640625 3.046875 -6.640625 C 1.765625 -6.640625 0.421875 -5.359375 0.421875 -3.15625 C 0.421875 -0.484375 1.578125 0.21875 2.5 0.21875 C 3.609375 0.21875 4.5625 -0.71875 4.5625 -2.03125 Z M 3.65625 -2.046875 C 3.65625 -1.5625 3.65625 -1.0625 3.484375 -0.703125 C 3.1875 -0.109375 2.734375 -0.0625 2.5 -0.0625 C 1.875 -0.0625 1.578125 -0.65625 1.515625 -0.8125 C 1.328125 -1.28125 1.328125 -2.078125 1.328125 -2.25 C 1.328125 -3.03125 1.65625 -4.03125 2.546875 -4.03125 C 2.71875 -4.03125 3.171875 -4.03125 3.484375 -3.40625 C 3.65625 -3.046875 3.65625 -2.53125 3.65625 -2.046875 Z M 3.65625 -2.046875 "/>
 
 
<path style="stroke:none;" d="M 4.5625 -1.671875 C 4.5625 -2.03125 4.453125 -2.484375 4.0625 -2.90625 C 3.875 -3.109375 3.71875 -3.203125 3.078125 -3.609375 C 3.796875 -3.984375 4.28125 -4.5 4.28125 -5.15625 C 4.28125 -6.078125 3.40625 -6.640625 2.5 -6.640625 C 1.5 -6.640625 0.6875 -5.90625 0.6875 -4.96875 C 0.6875 -4.796875 0.703125 -4.34375 1.125 -3.875 C 1.234375 -3.765625 1.609375 -3.515625 1.859375 -3.34375 C 1.28125 -3.046875 0.421875 -2.

In [48]:
for (cur_key, cur_value) in work_dict
    println(cur_key)
    println((mean(cur_value)-minimum(cur_value))/(maximum(cur_value)-minimum(cur_value)))
end

PL
0.25734957212842346
WTOT
0.21894416311077547
BT
0.27970400793013894
table_tau
0.2639063334034997
IP
0.33209939490606494
PRAD
0.19432534247458122
PREAL
0.25734957212842346
NG
0.34039691946501377
AMIN
0.5950981004768007
MEFF
0.4848515507147225
fit_tau
0.2593769165307266
EPSILON
0.6181399074501184
SELDB3
NaN
NEL
0.09465726629487649
TEV
0.18204209887096567
KAREA
0.40250870010853756
RMAG
0.6514967707127883


In [49]:
for (cur_key, cur_value) in work_dict
   println(cur_key) 
    println(minimum(cur_value)/(maximum(cur_value)-minimum(cur_value)))
end

PL
0.015371540177659373
WTOT
0.0006100184346554133
BT
0.15341080591608813
table_tau
0.010578012844729883
IP
0.03206352397225852
PRAD
0.0
PREAL
0.015371540177659373
NG
0.14619395549162512
AMIN
0.2280891581908678
MEFF
0.5030181086519114
fit_tau
0.009927533010896182
EPSILON
0.6802245614191905
SELDB3
Inf
NEL
0.029184383455740242
TEV
0.010110572553771584
KAREA
0.656800707627407
RMAG
0.24918602127197745


In [27]:
> 0.0084

PREAL - -0.5
RMAG - 0.5

BT - 0.1
KAREA - 0.1

EPSILON - 0.04
MEFF - 0.06
TEV - 0.1

IP - 0.7


LoadError: [91msyntax: ">" is not a unary operator[39m